### IMPORT PACKAGES

In [ ]:
!pip3 install  xlrd==1.2.0
!pip install --upgrade pandas
!pip install underthesea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import time
import pandas as pd
from pandas import Series, DataFrame
import xlrd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score
from sklearn.feature_extraction.text import TfidfVectorizer
from xlrd import open_workbook
from underthesea import word_tokenize


### TASK 1: TOXIC COMMENT RECOGNIZATION ON LIVE FOOTBALL ON YOUTUBE

#### 2.1 Read data from FLORENTINO

In [ ]:

dataTrainUIT = open_workbook("/content/train.xlsx", encoding_override='latin')
dataTrainUIT = pd.read_excel(dataTrainUIT)

dataValidUIT = open_workbook("/content/valid.xlsx", encoding_override='latin')
dataValidUIT = pd.read_excel(dataValidUIT)
# 693
data_TestUIT = open_workbook("/content/test.xlsx", encoding_override='latin')
data_TestUIT = pd.read_excel(data_TestUIT)
# Model
model = DecisionTreeClassifier()

trainY = dataTrainUIT.Final
validY = dataValidUIT.Final
testY  = data_TestUIT.Final

dataTrainUIT.head()


,Content,Final
0,cặc,1
1,da kieu gi may dua,0
2,cố lên vn,0
3,cl mệ nó,1
4,Việt Nam ăn chắc rồi🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳,0


In [ ]:
dataList1 = open_workbook("/content/data1.xlsx", encoding_override='latin')
dataList1 = pd.read_excel(dataList1)

dataList2 = open_workbook("/content/data2.xlsx", encoding_override='latin')
dataList2 = pd.read_excel(dataList2)
# 693
dataList3 = open_workbook("/content/data3.xlsx", encoding_override='latin')
dataList3 = pd.read_excel(dataList3)

#### 2.2 Text preprocess (stopword, word segmentation, learn vocabulary and idf)

In [ ]:
def text_process(datasets):
    # Stopwords
    StopWords = [
        'chs','cerrrr','aaaaa','aaaaaaa','aamir','abcxyz','ac','18','200','500','dek','thg','đg','đs','đm','đuma',
        'vl','vcl','kkk','dcm','cu','ừm','đĩ','đụ','địt','xl','lol','01','10','100', '11','12','13','14','15','150',
        '17','1700''1967','20','21','22','225','23','24','25','26','28','2_','2_3','30','300','3000','320','333',
        '33333','40','400','42','45','48', '50','5000','580','60','63','66','75','78','80','800','81','850','90','900',
        '99','99999','_200','_5','ah','bn','c3','chg','cp','dòg','hlin','lòn','lôz','tđ','đkm','đkmm','đmaaaa','đmm',
        'đmmmmm','đỹ','vcb','vclll','đụ_mẹ','trươ','trưen','amir','ga','1700','1967','bg','chaiii','clm','cmm','cmnl','cã',
        'hloz','imdb','kau','kbh','loz','lozzz','lozzzz','matlon','muô','nh','nhma','p30','16','250','56','adm','ngươ'
    ]
    Tfidf = TfidfVectorizer(stop_words = StopWords)
    
    # Word Segmentation
    WordSeg = []
    for i in range(datasets.shape[0]):
        sentence_strip = str(datasets.Content[i]).strip()# Remove spaces at begin & end
        WordSeg.append(word_tokenize(sentence_strip, format='text')) # format text: add "_" where text is phrases.
    datasets.Content = WordSeg # Update new sentences
    
    # Learn vocabulary and idf.
    X = Tfidf.fit_transform(datasets.Content)
    # Display after processing
    print(Tfidf.get_feature_names()); print("\n\n")
    # return document-term matrix after processing
    return X


#### 2.3 Using Train&Valid sets of FLORENTINO for training Toxic Comment Recognization model

In [ ]:
trainY = pd.Series(data=dataTrainUIT.Final).to_frame()
validY = pd.Series(data=dataValidUIT.Final).to_frame()

trainValidX = pd.concat([dataTrainUIT, dataValidUIT]).reset_index(drop=True)
print(trainValidX)



                                                Content  Final
0                                                   cặc      1
1                                    da kieu gi may dua      0
2                                             cố lên vn      0
3                                              cl mệ nó      1
4                        Việt Nam ăn chắc rồi🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳      0
...                                                 ...    ...
1393                          bắt del công bằng cc j cả      1
1394                                        vn đá wa do      0
1395                             đá gãy chân a tài luôn      1
1396                             trọng tài thiên vị vcl      1
1397  FiFa sinh ra 1 lũ ngồi xem va làm gì cho tốn c...      1

[1398 rows x 2 columns]


In [ ]:
trainValidX = text_process(trainValidX)
trainValidY = pd.concat([trainY, validY]).reset_index(drop=True)

trainX, testX, trainY, testY = train_test_split(trainValidX, trainValidY, test_size=0.33, random_state=42)
model = model.fit(trainX, trainY)
predY = model.predict(testX)

['0000000000000000', '02', '08', '0_', '0_2', '0_saudi_arabia', '111111111111111110', '1234567890', '19', '1_0_arp', '1_4', '1_nghiêng', '27', '2_1', '32', '3_', '3_tỉ', '87', '90_', '99999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999', '_1', '_9', '___', '_a', '_ae', '_ai', '_chat', '_check', '_chuyền', '_dcmmm', '_dคｙ', '_hõ', '_lẽ_ra', '_mu', '_muộn', '_ok', '_trang', '_tóm_lại', '_việt', '_vn', '_ông', '_đá', '_đê', '_đội_hình', '_๖_', '_ủa', 'a_tài', 'aaaaaaaaaaaaaaaaaaa', 'acc', 'accmai_bay', 'ad', 'add', 'admin', 'adu', 'ae', 'ai', 'ala', 'all', 'an', 'anh', 'anh_em', 'anh_tài', 'anhyêu_tố', 'anti', 'ao', 'ap', 'arap', 'arapxeut', 'arat', 'avai', 'ay', 'bai', 'balo', 'ban', 'ban_đầu', 'bang', 'banh', 'bao', 'bao_giờ', 'barcelona', 'bat', 'bay', 'bi', 'bik', 'biên', 'biết', 'bl', 'bluan', 'bluan_viên', 'blv', 'bnaf', 'bo', 'bon', 'bong_da', 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


#### 2.4 Performance metrics (Accuracy, F1-score…) for the test sets in UIT-VSMEC

In [ ]:
Accuracy  = []
Precision = []
F1_Score  = []

Accuracy.append(accuracy_score(testY, predY))
F1_Score.append(f1_score(testY, predY, average='weighted'))  # Could change to None,'weighted','micro','macro'
Precision.append(precision_score(testY, predY, average='weighted'))

averageAccuracy = sum(Accuracy)/len(Accuracy)
print("Average of Accuracy:", averageAccuracy)
averageF1_Score = sum(F1_Score)/len(F1_Score)
print("Average of F1_Score:", averageF1_Score)
averagePrecision = sum(Precision)/len(Precision)
print("Average of Precision:", averagePrecision)

Average of Accuracy: 0.803030303030303
Average of F1_Score: 0.802731728239629
Average of Precision: 0.8046183881921197


### TASK 3: APPLYING THE TRAINED MODEL IN TASK 2 TO THREE DATASETS IN TASK 1

#### 3.1 Applying the trained model in Task 2 to three datasets in Task 1

In [ ]:
dataList1.head()

,Content,Final
0,lại tạt,0
1,Thua đổ trọng tài hồi nào địt mẹ thằng kia,1
2,Nhục nhã,0
3,thua rồi đá làm gì nữa,0
4,xin chào,0


In [ ]:
dataList2.head()


,Content,Final
0,2 quả thua lãng xẹt ả rập trc khi vào quả đầu ...,0
1,"nhìn kiểu chuyền bóng cứ ngơ ngơ, ko có liên l...",0
2,đá thì đá nhanh đi câu làm cc j nữa,1
3,tim ny kk,0
4,Ncc,1


In [ ]:
dataList3.head()

,Content,Final
0,tao dái 2-1 cho mấy thằng cá độ thua đi,1
1,sút thế kia đéo ăn thẻ thì ăn gì ?,1
2,quá ghê gớmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm...,0
3,Xin hùng dũng 30k,0
4,Tiến Linh ko làm đc gì hết,0


In [ ]:
dataListY1 = dataList1.Final
dataListY2 = dataList2.Final
dataListY3 = dataList3.Final

dataListX1 = text_process(dataList1)
trainX1, testX1, trainY1, testY1 = train_test_split(dataListX1, dataListY1, test_size=0.33, random_state=42)
model = model.fit(trainX1, trainY1)
predY1 = model.predict(testX1)

dataListX2 = text_process(dataList2)
trainX2, testX2, trainY2, testY2 = train_test_split(dataListX2, dataListY2, test_size=0.33, random_state=42)
model = model.fit(trainX2, trainY2)
predY2 = model.predict(testX2)

dataListX3 = text_process(dataList3)
trainX3, testX3, trainY3, testY3 = train_test_split(dataListX3, dataListY3, test_size=0.33, random_state=42)
model = model.fit(trainX3, trainY3)
predY3 = model.predict(testX3)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['00', '0000000', '0_3', '123345689123456789123456789123456789123456789123456789123456789123456789', '1_', '1_0', '2_0', '2_1', '2p', '44', '5p', '_0', '_2', '_còn', '_dit_me', '_ko', '_loai', '_làm', '_mấy', '_não', '_nói', '_nổ', '_phạt', '_thua', '_thủ_môn', '_đội', 'acc', 'accc', 'ae', 'ai', 'akari', 'al', 'am', 'an', 'anh', 'anti', 'ao', 'araki', 'arap', 'av', 'bai', 'balo', 'ban', 'bang', 'banh', 'bao_nhiêu', 'bay', 'be', 'bi', 'biet', 'binh', 'biên_cương', 'biến', 'biết', 'blv', 'bome', 'bon', 'bt', 'btg', 'buồi', 'buồn_ngủ', 'buổi', 'bài', 'bàn', 'bàn_thờ', 'bán', 'bán_độ', 'bây', 'bé', 'béo', 'béomẹ', 'bên', 'bình_luận', 'bình_thường', 'bìu', 'bí_thư', 'bò', 'bóng', 'bóng_đá', 'bùi', 'bú', 'bướm_đêm', 'bười', 'bạn', 'bảng', 'bảo', 'bất_đắc_dĩ', 'bẩn', 'bắt', 'bắt_lỗi', 'bằng', 'bẻ', 'bị', 'bịp', 'bọn', 'bỏ_mẹ', 'bố', 'bốc', 'bốc_phét', 'bổn', 'bớt', 'bực', 'ca', 'ca1ig', 'cac', 'cai', 'cal', 'campuchia', 'canh', 'cao', 'cat', 'cau', 'cayyyyyy', 'cc', 'ccc', 'cccccccccccccccccc

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['00', '0000001', '000_', '0_', '0_dược', '1000', '111111121', '1_0', '1ksa', '3223', '46', '4g', '_000', '_1', '_chả', '_dit_me', '_dlm', '_dễ', '_giỏi', '_kkkk', '_mấy', '_nên', '_san', '_vn', '_ép', '_đm', '_đá', '_đừng', 'acc', 'adu', 'ae', 'aesw', 'ahihi', 'ai', 'an', 'an_toàn', 'anh', 'anh_em', 'ao', 'arap2', 'arapxi', 'ay', 'ay_bi_sao', 'ba', 'ba_que', 'bai', 'ban', 'banh', 'bao_nhiêu', 'bat', 'bay', 'bexuanmailonto', 'bi', 'binh', 'bip', 'biết', 'biển', 'bl', 'black', 'blv', 'bon', 'bong', 'brazil', 'bucu', 'buoi', 'buồi', 'buồn', 'buồn_cười', 'bà', 'bà_con', 'bài', 'bàn', 'bàn_thờ', 'bàng', 'bán', 'bán_kết', 'bán_nước', 'bán_độ', 'bão', 'bên', 'bình_luận', 'bình_luận_viên', 'bình_định', 'bìuuuuuuuuuuuuuuuuuuuuuuuuuuu', 'bóng', 'bóng_vàng', 'bóng_đá', 'bùi', 'bú', 'băt', 'bơi', 'bướm', 'bạn', 'bản', 'bản_mặt', 'bảo', 'bảo_thắng', 'bảo_thủ', 'bất_lực', 'bẩn', 'bẩn_thỉu', 'bận', 'bật', 'bắn', 'bắt', 'bắt_lỗi_việt_nam', 'bắt_ép', 'bằng', 'bị', 'bọn', 'bỏ', 'bố', 'bổn', 'bộ', 'bớt'

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


#### 3.2 Performance metrics (Accuracy, F1-score…) for these datasets

In [ ]:
Accuracy1  = []
Precision1 = []
F1_Score1  = []

Accuracy1.append(accuracy_score(testY1, predY1))
F1_Score1.append(f1_score(testY1, predY1, average='micro'))  # Could change to None,'weighted','micro','macro'
Precision1.append(precision_score(testY1, predY1, average='micro'))

averageAccuracy1 = sum(Accuracy1)/len(Accuracy1)
print("Average of Accuracy 1:", averageAccuracy1)
averageF1_Score1 = sum(F1_Score1)/len(F1_Score1)
print("Average of F1_Score 1:", averageF1_Score1)
averagePrecision1 = sum(Precision1)/len(Precision1)
print("Average of Precision 1:", averagePrecision1)

Average of Accuracy 1: 0.7489177489177489
Average of F1_Score 1: 0.7489177489177489
Average of Precision 1: 0.7489177489177489


In [ ]:
Accuracy2  = []
Precision2 = []
F1_Score2  = []

Accuracy2.append(accuracy_score(testY2, predY2))
F1_Score2.append(f1_score(testY2, predY2, average='micro'))  # Could change to None,'weighted','micro','macro'
Precision2.append(precision_score(testY2, predY2, average='micro'))

averageAccuracy2 = sum(Accuracy2)/len(Accuracy2)
print("Average of Accuracy 2:", averageAccuracy2)
averageF1_Score2 = sum(F1_Score2)/len(F1_Score2)
print("Average of F1_Score 2:", averageF1_Score2)
averagePrecision2 = sum(Precision2)/len(Precision2)
print("Average of Precision 2:", averagePrecision2)

Average of Accuracy 2: 0.7660377358490567
Average of F1_Score 2: 0.7660377358490567
Average of Precision 2: 0.7660377358490567


In [ ]:
Accuracy3  = []
Precision3 = []
F1_Score3  = []

Accuracy3.append(accuracy_score(testY3, predY3))
F1_Score3.append(f1_score(testY3, predY3, average='micro'))  # Could change to None,'weighted','micro','macro'
Precision3.append(precision_score(testY3, predY3, average='micro'))

averageAccuracy3 = sum(Accuracy3)/len(Accuracy3)
print("Average of Accuracy 3:", averageAccuracy2)
averageF1_Score3 = sum(F1_Score3)/len(F1_Score3)
print("Average of F1_Score 3:", averageF1_Score3)
averagePrecision3 = sum(Precision3)/len(Precision3)
print("Average of Precision 3:", averagePrecision3)

Average of Accuracy 3: 0.7660377358490567
Average of F1_Score 3: 0.7186147186147186
Average of Precision 3: 0.7186147186147186
